In [1]:
!ls "/Users/ijeonghwan/Dev/github/pytorch/"

BUCK.oss             android              mypy.ini
BUILD.bazel          aten                 mypy_plugins
CITATION.cff         aten.bzl             pt_ops.bzl
CMakeLists.txt       benchmarks           pt_template_srcs.bzl
CODEOWNERS           binaries             pyproject.toml
CODE_OF_CONDUCT.md   buckbuild.bzl        pytest.ini
CONTRIBUTING.md      build.bzl            requirements.txt
Dockerfile           build_variables.bzl  scripts
GLOSSARY.md          c10                  setup.py
LICENSE              caffe2               test
MANIFEST.in          cmake                third_party
Makefile             defs.bzl             tools
NOTICE               docker.Makefile      torch
README.md            docs                 torchgen
RELEASE.md           functorch            ubsan.supp
SECURITY.md          ios                  ufunc_defs.bzl
WORKSPACE            mypy-strict.ini      version.txt


In [37]:
code_path = "/Users/ijeonghwan/Dev/github/pytorch/test"
rst_path = "/Users/ijeonghwan/Dev/github/pytorch/docs"

In [38]:
# .rst : reStructuredText, python docs 작성에 사용되는 마크업 구문으로 정의, 도큐먼트 형식으로 로드
from langchain_community.document_loaders import TextLoader
import os

rst_documents = []
for dirpath, dirnames, filenames in os.walk(rst_path):
    for file in filenames:
        if (file.endswith(".rst")):
            try:
                loader = TextLoader(os.path.join(
                    dirpath, file), encoding="utf-8")
                rst_documents.extend(loader.load())

            except Exception:
                pass

print(f".rst 파일의 개수: {len(rst_documents)}")

.rst 파일의 개수: 195


In [39]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language


py_documents = []

loader = GenericLoader.from_filesystem(
    code_path,
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(
        language=Language.PYTHON, parser_threshold=30
    ),
)
py_documents.extend(loader.load())

print(f".py 파일의 개수: {len(py_documents)}")

.py 파일의 개수: 5286


In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

py_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)

py_docs = py_splitter.split_documents(py_documents)

print(f"분할된 .py 파일의 개수: {len(py_docs)}")

rst_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)

rst_docs = rst_splitter.split_documents(rst_documents)

print(f"분할된 .rst 파일의 개수: {len(rst_docs)}")

분할된 .py 파일의 개수: 19563
분할된 .rst 파일의 개수: 1084


In [27]:
combined_documents = py_docs + rst_docs
print(f"총 도큐먼트 개수: {len(combined_documents)}")

총 도큐먼트 개수: 20647


In [31]:
from dotenv import load_dotenv

load_dotenv()

True

In [36]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

# print(f"[API KEY] : {os.environ['OPENAI_API_KEY']}")

In [41]:
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# Create a LocalFileStore instance to use local file storage
store = LocalFileStore("./cache/")

# Create a OpenAI embedding model instance
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", disallowed_special=())

# Cache Embedding:
# Cache the embedding calculation results using CacheBackedEmbeddings
# We can reuse values ​​calculated once without having to calculate the embedding multiple times
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings, 
    document_embedding_cache=store, 
    namespace=embeddings.model
)

In [43]:
from langchain_community.vectorstores import FAISS

# Specify a folder name for the faiss index to be stored locally
FAISS_DB_INDEX = "langchain_faiss"

# Create a FAISS DB instance
db = FAISS.from_documents(combined_documents, cached_embeddings)

# Save the created database instance locally in a specified folder
db.save_local(folder_path=FAISS_DB_INDEX)

In [45]:
# There is no need to redo the cells above
from langchain_community.vectorstores import FAISS

# Load vector index
db = FAISS.load_local(
    FAISS_DB_INDEX, # Directory name of the FAISS index to load
    cached_embeddings, # Provides embedding information
    allow_dangerous_deserialization=True,
)

In [46]:
# Creates a VectorStoreRetriever object based on the current vector store
# 2000 x 10
faiss_retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 10})

In [48]:
# Import the bm25Retriever class from the langchain.retrievers module
from langchain.retrievers import BM25Retriever

# Create a BM25 retriever model instance using the document collection
bm25_retriever = BM25Retriever.from_documents(
    combined_documents # document collection for initialization
)

# Set the k property of the BM25Retriever instance to 10 and return up to 10 results when retrieved.
bm25_retriever.k = 10

In [49]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.6, 0.4],
    search_type="mmr", # MMR method that improves the diversity of search results
)

In [50]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 20년차 AI 개발자입니다. 당신의 임무는 주어진 질문에 대하여 최대한 문서의 정보를 활용하여 답변하는 것입니다.
    문서는 Python 코드에 대한 정보를 담고 있습니다. 따라서 답변을 작성할 때에는 Python 코드에 대한 상세한 code snippet을
    포함하여 작성해주세요. 최대한 자세하게 답변하고, 한글로 답변해 주세요. 주어진 문서에서 답변을 찾을 수 없는 경우, "문서에
    답변이 없습니다."라고 답변해 주세요. 답변은 출처(source)를 반드시 표기해 주세요.
    
    #참고문서:
    {context}

    #질문:
    {question}

    #답변:

    출처:
    - source1
    - source2
    - ...
    """
)

In [53]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.runnables import ConfigurableField
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)

llm = ChatOpenAI(
    model="gpt-4-turbo-preview",
    temperature=0,
    streaming=True,
    callbacks=[StreamCallback()],
).configurable_alternatives(
    ConfigurableField(id="llm"),
    default_key="gpt4",
    ollama=ChatOllama(
        model="llama3-instruct-8b:latest",
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    ),
    gpt3=ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        streaming=True,
        callbacks=[StreamCallback()],
    ),
)

In [54]:
rag_chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [56]:
answer = rag_chain.with_config(configurable={"llm": "gpt4"}).invoke(
    "torch.autograd 사용 방법을 알려주세요"
)

`torch.autograd`는 PyTorch에서 자동 미분을 제공하는 기능으로, 신경망 학습 시에 역전파를 통해 파라미터의 그래디언트를 자동으로 계산해줍니다. 사용자는 `torch.Tensor` 객체의 `.backward()` 메소드를 호출하여 그래디언트를 자동으로 계산할 수 있으며, 이를 통해 모델의 가중치를 업데이트할 수 있습니다.

기본적인 사용 방법은 다음과 같습니다:

1. 모델의 파라미터를 `requires_grad=True`로 설정하여 해당 텐서에 대한 그래디언트 계산이 필요함을 명시합니다.
2. 순전파(forward pass)를 실행하여 출력값을 계산합니다.
3. 손실 함수(loss function)를 계산합니다.
4. `.backward()`를 호출하여 역전파(backward pass)를 실행하고, 각 파라미터의 그래디언트를 자동으로 계산합니다.
5. 그래디언트를 사용하여 모델의 파라미터를 업데이트합니다.

아래는 간단한 예제 코드입니다:

```python
import torch

# 입력 텐서 x와 파라미터 w, b를 정의하고, requires_grad=True로 설정하여 그래디언트 계산이 필요함을 명시합니다.
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
w = torch.tensor([2.0], requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)

# 순전파: 모델의 예측값 y를 계산합니다.
y = w * x + b

# 손실 함수: 예측값 y와 실제값과의 차이를 제곱하여 손실을 계산합니다.
loss = torch.sum((y - torch.tensor([2.0, 4.0, 6.0])) ** 2)

# 역전파: 손실 함수의 그래디언트를 계산합니다.
loss.backward()

# 그래디언트 출력
print(f'x.grad: {x.grad}, w.grad: {w.grad}, b.grad: {b.grad}')

# 그래디언트를 

In [57]:
answer = rag_chain.with_config(configurable={"llm": "gpt4"}).invoke(
    "torch.cuda 에 대해서 알려주세요"
)

`torch.cuda`는 PyTorch에서 NVIDIA CUDA를 사용하여 GPU 상에서 텐서 연산을 가속화하기 위한 모듈입니다. 이 모듈을 통해 GPU에서의 계산을 관리하고 최적화할 수 있는 다양한 기능과 도구를 제공합니다. 여기에는 GPU 장치 관리, 메모리 관리, 스트림 및 이벤트 관리 등이 포함됩니다.

주요 기능들은 다음과 같습니다:

- `torch.cuda.is_available()`: CUDA가 사용 가능한지 여부를 반환합니다. 이는 GPU가 시스템에 설치되어 있고 PyTorch가 CUDA를 사용할 수 있는 상태인지 확인하는 데 사용됩니다.
- `torch.cuda.device_count()`: 사용 가능한 GPU 장치의 수를 반환합니다.
- `torch.cuda.set_device(device)`: 기본 GPU 장치를 설정합니다.
- `torch.cuda.current_device()`: 현재 활성화된 GPU 장치의 인덱스를 반환합니다.
- `torch.cuda.memory_allocated()`: 현재 할당된 메모리의 양을 반환합니다.
- `torch.cuda.memory_reserved()`: 현재 예약된(총 할당 가능한) 메모리의 양을 반환합니다.
- `torch.cuda.synchronize()`: 현재 선택된 GPU 장치의 모든 스트림을 동기화합니다.

또한, `torch.cuda.Stream`을 사용하여 비동기 연산을 관리할 수 있으며, `torch.cuda.Event`를 통해 연산의 시작과 끝을 표시하여 성능 측정에 사용할 수 있습니다.

CUDA 메모리 관리에 관련된 더 세부적인 기능으로는 메모리 할당, 해제 및 최적화를 위한 도구들이 있습니다. 예를 들어, `torch.cuda.empty_cache()`를 호출하여 불필요한 메모리를 해제하고 GPU 메모리 사용량을 최적화할 수 있습니다.

이러한 기능들은 GPU를 사용하여 딥러닝 모델을 훈련시키거나, 대규모 텐서 연산을 수행할 때 중요한 역할을 합니다. PyTorch의 `torc

In [62]:
answer = rag_chain.with_config(configurable={"llm": "ollama"}).invoke(
    "torch optim에 대해서 알려주세요"
)

`torch.optim`은 PyTorch에서 모델의 파라미터를 최적화하기 위해 사용되는 모듈입니다. 이 모듈은 다양한 최적화 알고리즘을 제공하여, 모델 학습 시 그래디언트(기울기)를 기반으로 파라미터를 업데이트하는 과정을 쉽게 만들어 줍니다.

`torch.optim`을 사용하기 위해서는 먼저 최적화할 모델의 파라미터와 함께 최적화 알고리즘을 선택해야 합니다. 그 후, `.step()` 메소드를 호출하여 파라미터를 업데이트합니다. 이 과정은 일반적으로 손실 함수의 그래디언트를 계산한 후에 이루어집니다.

예를 들어, SGD(Stochastic Gradient Descent) 최적화 알고리즘을 사용하는 경우 다음과 같이 최적화 객체를 생성할 수 있습니다:

```python
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
```

여기서 `model.parameters()`는 최적화할 모델의 파라미터를 나타내고, `lr`은 학습률(learning rate)을, `momentum`은 모멘텀 값을 의미합니다.

또한, `torch.optim`은 파라미터 그룹별로 다른 최적화 옵션을 지정할 수 있도록 지원합니다. 예를 들어, 모델의 일부 파라미터에는 더 높은 학습률을 사용하고 싶을 때 다음과 같이 설정할 수 있습니다:

```python
optimizer = optim.SGD([
    {'params': model.base.parameters(), 'lr': 1e-2},
    {'params': model.classifier.parameters()}
], lr=1e-3, momentum=0.9)
```

이 코드는 `model.base`의 파라미터에는 `1e-2`의 학습률을, `model.classifier`의 파라미터에는 기본 학습률인 `1e-3`을 사용하도록 설정합니다. 모든 파라미터 그룹에 대해 `momentum=0.9`가 적용됩니다.

출